<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [556]:
import pandas as pd
import psycopg2
import requests 
from bs4 import BeautifulSoup 
# Для графиков
import plotly.express as px

# Данная библиотека импортируется ИСКЛЮЧИТЕЛЬНО для возможности отображения графиков через
# https://nbviewer.org/ Jupyter Notebook. Для действий над данными, НЕ ИСПОЛЬЗУЕТСЯ!!!
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio

In [557]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [558]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [559]:
# текст запроса
query_3_1 = f'''
    SELECT count(*) FROM vacancies;
'''


In [560]:
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\4206390918.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [561]:
# текст запроса
query_3_2 = f'''
    SELECT count(*) FROM employers;
'''

In [562]:
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\3994507252.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [563]:
query_3_3 = f'''
SELECT count(*) FROM areas;
'''

In [564]:
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\2361023096.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [565]:
query_3_4 = f'''
SELECT count(*) FROM industries;
'''

In [566]:
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\1485325140.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,294


***

В исследуемой БД:
49197 вакансий
23501 работодателей
работу предлагают в 1362 регионах
поиск работы охватывает 294 сферы деятельности

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [567]:
# текст запроса
query_4_1 = f'''
SELECT 
	a."name" as "регион", 	
	count(v.area_id) as "количество вакансий"
FROM vacancies v
join areas a on v.area_id = a.id 
group by "регион"
order by "количество вакансий" desc 
'''

In [568]:
df = pd.read_sql_query(query_4_1, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\168200814.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,регион,количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [569]:
query_4_2 = f'''
select count(*) from vacancies v
where (salary_from is not null) or (salary_to is not null)
'''

In [570]:
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\2461021528.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [571]:
query_4_3 = f'''
select 
	ROUND(avg(salary_from),0) "ср. для нижней границы ЗП ",
	ROUND(avg(salary_to),0) "ср. для верхней границы ЗП "
from vacancies v
'''

In [572]:
df = pd.read_sql_query(query_4_3, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\414995271.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,ср. для нижней границы ЗП,ср. для верхней границы ЗП
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [573]:
query_4_4 = f'''
select 
	schedule,
	employment,
	count(*) vacancie_cnt
from vacancies v
group by schedule, employment
order by 3 desc 
'''

In [574]:
df = pd.read_sql_query(query_4_4, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\1428108846.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,vacancie_cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [575]:
# текст запроса
query_4_5 = f'''
select 
	experience ,
	count(*) 
from vacancies v
group by experience
order by 2
'''

In [576]:
df = pd.read_sql_query(query_4_5, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\110506269.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


ВЫВОДЫ:
1) Лидерами по количеству вакансий являются города: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. Это не удивительно, т.к. именно там больше всего население
2) У больше половины вакансий не заполнены поля с желаемой зарплатой. Заполнено всего у 24073 из 49197
3) Зарплатная вилка в среднем не сильно большая от 71065 до 110537
4) У большинства соискателей в требованиях указаны Полный день и Полная занятость
5) Большинство соискателей имеют опыт работы от 1 до 3 лет 
6) Из графика видно, что при частичной занятости предлагается в большенстве случаев удаленная работа

***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [577]:
# текст запроса
query_5_1 = f'''
select
	e."name" as "работодатель",
	count(*) as "количество вакансий"
from vacancies v
join employers e on v.employer_id = e.id
group by "работодатель"
order by "количество вакансий" desc
limit 5
'''

In [578]:
df = pd.read_sql_query(query_5_1, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\3617394740.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,работодатель,количество вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [579]:
query_5_2 = f'''
SELECT 
	a."name"  as "регион", 	
	count(e.id) as "кол. работодателей без вакансий"
FROM employers e 
	left join vacancies v on e.area =v.area_id
	join areas a on a.id =e.area  
group by "регион"
having count(v.id) = 0
order by count(e.id) desc 
limit 5
'''

In [580]:
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\714344905.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,регион,кол. работодателей без вакансий
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Беларусь,18


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [581]:
query_5_3 = f'''
select 
	e."name" as "работодатель", 	
	count(distinct v.area_id) as "количество регионов"
from  employers e 
join vacancies v on e.id = v.employer_id 
group by e.id 
order by "количество регионов" desc 
limit 5
'''

In [582]:
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\2060984172.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,работодатель,количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [583]:
# текст запроса
query_5_4 = f'''
select 
	count(e.id) "кол. работодателей без сферы деятельности"
from employers e 
left join employers_industries ei on e.id = ei.employer_id 
where ei.employer_id is null
'''

In [584]:
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\3677090595.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,кол. работодателей без сферы деяте
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [585]:
query_5_5 = f'''
select 
	e."name" as "название компании",	
	count(ei.industry_id) as "количество сфер деятельности"
from employers e 
join employers_industries ei on e.id = ei.employer_id 
group by e.id 
having count(ei.industry_id) = 4
order by "название компании"
limit 3
'''

In [586]:
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\2154720907.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,название компании,количество сфер деятельности
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [587]:
query_5_6 = f'''
select 
	count(e.id) as "сфера деятельности - разработка ПО"
from employers_industries ei  
join employers e  on e.id = ei.employer_id 
join industries i on ei.industry_id = i.id 
group by i."name" 
having i."name" = 'Разработка программного обеспечения'
'''

In [588]:
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\2982424571.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,сфера деятельности - разработка ПО
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [589]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' # Определяем адрес страницы
response = requests.get(url, verify=False)
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup
table = page.find('table',class_='standard sortable')
links = table.find_all('a')
# Извлечем данные из таблицы
table_data = []
for row in table.find_all('tr'):
    columns = row.find_all('td')[1:2] # берем только колонку, где указан город
    if columns:
        table_data.append(columns[0].text.strip()) # добавляем город в список
cities = "','".join(table_data)
cities = "'" + cities + "'"
cities

C:\Users\NCHirkov\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'ru.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings



"'Москва','Санкт-Петербург','Новосибирск','Екатеринбург','Казань','Красноярск','Нижний Новгород','Челябинск','Уфа','Самара','Ростов-на-Дону','Краснодар','Омск','Воронеж','Пермь','Волгоград'"

In [590]:
# текст запроса
query_5_7 = f'''
select
	a."name" as "регион",
	count(v.id) as "количество вакансий"
from vacancies v
join employers e on v.employer_id = e.id
join areas a on a.id = v.area_id 
where e."name" = 'Яндекс' and a."name" in ({cities})
group by e."name", "регион"

union 

select
	'total',
	count(v.id)
from vacancies v
join employers e on v.employer_id = e.id
join areas a on a.id = v.area_id 
where e."name" = 'Яндекс' and a."name" in ({cities})
order by "количество вакансий"
'''


In [591]:
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\1825505273.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,регион,количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


**ВЫВОДЫ:**
1) Возглавляет список работодателей, у которых больше всего опубликовано вакансий Яндекс. Опубликовано 1933 ваканcии. На втором и третьем месте это Ростелеком и Тинькофф
2) Россия, скорее всего из-за общего количества работодателей, возглавляет список регионов по количеству работодателей у которых нет вакансий (410). На втором месте Казахстан (207), а на третьем Московская область (75). Я бы обратил внимание на Московскую область и Краснодарский край, который на четвертом месте, т.к. по соотношению масштабов, можно сделать вывод, что в этих регионов хуже всего с поиском работы.
3) По количеству регионов, в которых опубликованы вакансии, на первом месте все тот-же Яндекс (181), на втором Ростелеком (152), а на третьем Спецремонт (116). Тут бы я сделал вывод, что такой охват регионов может быть связан с возможностью удаленной работы или со спецификой деятельности, как в Спецремонт.
4) Только у 15% (3553) из 23501 сфера деятельности указана "Разработка программного обеспечения" 


***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [592]:
query_6_1 = f'''
select
	count(v.id)	"кол. вакансий имеет отн. к данным"
from vacancies v
where lower(v."name") like '%data%' or lower(v."name") like '%данн%'
'''

In [593]:
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\950281682.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,кол. вакансий имеет отн. к данным
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [594]:
vacancies=['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%']
vacancies_str = "'" + "', '".join(vacancies) + "'"

query_6_2 = f'''
select 
	count(v.id) "кол. вакансий DS"
from vacancies v 
where (v."name"  ilike any (array[{vacancies_str}]) 
	or (v."name" not ilike '%HTML%' and v."name" ilike '%ML%'))
	and (v."name" ilike '%junior%' or v.experience = 'Нет опыта' or v.employment = 'Стажировка')
'''

In [595]:
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\2008915662.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,кол. вакансий DS
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [596]:
query_6_3 = f'''
select 
	count(v.id) "кол. вакансий DS с навыками SQL, postgres"
from vacancies v 
where (v."name"  ilike any (array[{vacancies_str}]) 
	or (v."name" not ilike '%HTML%' and v."name" ilike '%ML%'))
	and (v.key_skills ilike '%SQL%' or v.key_skills ilike '%postgres%')
'''

In [597]:
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\3268897861.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,"кол. вакансий DS с навыками SQL, postgres"
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [598]:
query_6_4 = f'''
select 
	count(v.id) "кол. вакансий DS с навыками Python"
from vacancies v 
where (v."name"  ilike any (array[{vacancies_str}]) 
	or (v."name" not ilike '%HTML%' and v."name" ilike '%ML%'))
	and (v.key_skills ilike '%Python%')
'''

In [599]:
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\463562881.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,кол. вакансий DS с навыками Python
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [600]:
query_6_5 = f'''
select 
	round(avg(length(key_skills)-length(replace(key_skills, CHR(9), ''))+1), 2) "ключевых навыков"
from vacancies v 
where (v."name"  ilike any (array[{vacancies_str}]) 
	or (v."name" not ilike '%HTML%' and v."name" like '%ML%'))
'''

In [601]:
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\2581522160.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,ключевых навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [602]:
query_6_6 = f'''
select 
	v.experience "опыт работы",	
	round(avg(coalesce ((v.salary_from + v.salary_to)/2 , v.salary_from, v.salary_to)),0) AS "ЗП"
from vacancies v 
where (v."name"  ilike any (array[{vacancies_str}]) 
	or (v."name" not ilike '%HTML%' and v."name" like '%ML%'))
	and (v.salary_from is not null or v.salary_to  is not null)
group by v.experience
'''

In [603]:
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\2835978903.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,опыт работы,ЗП
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


**ВЫВОД:**
1) 3,6% от общего числа вакансий (1771 из 49197), это вакансия имеет отношение к данным
2) Подходящих для начинающего дата-сайентиста всего 51 вакансия, а это 0,1% от всех вакансий и 2,9% от вакансий DS 
3) Вакансий DS (дата-сайентиста), в которых в качестве ключевого навыка указан SQL или postgres 229 (0,5% от всех и 12.9% от вакансий DS)
4) Вакансий DS (дата-сайентиста), в которых в качестве ключевого навыка указан Python 357 (0,7% от всех и 20.2% от вакансий DS)
5) В среднем, в вакансиях DS указано по 6 ключевых навыков
6) Предлагаемая ЗП в среднем:
- Без опыта - 74643
- От 1 года до 3 лет - 139675
- От 3 до 6 лет - 243115

***

# Общий вывод по проекту

**ОБЩИЙ ВЫВОД**

1) Вакансии для IT-специалистов возглавляют ТОП по отраслям и это не удивительно в век цифровизации.
2) Около 3,6% (1771) от общего числа вакансий имеют отношение к данным, но это цифра должна резко возрасти, т.к. есть общие тенденции использования инструментов ИИ во всех отраслях.
3) Также видно, что в крупных городах большое количество вакансий DS предпологает удаленную работу (Москва 45 вакансий из 169, Санкт-Петербург 11 вакансий из 49). В Воронеже и Екатеринбурге только вакансии с удаленной работой.
4) Не много, но есть на рынке предложения начинающим специалистам DS. Учитывая развития этой отрасли, количество предложений, где компании будут брать и развивать молодых специалистов, будет расти в ближайшие годы.
5) Уровень ЗП ИТ-специалистов в крупных городах выше среднего и как закономерность, наиболее высокие ЗП предлагают в крупных городах Москва, Новосибирск, Санкт-Петербург. Учитывая тенденции перехода на удаленную работу, разница ЗП крупных городов и регионов должна сгладиться.


## Дополнительные иследования

1. Интересно посмотреть, в каких сферах деятельности больше всего вакансий. ТОП 10.

In [604]:
# текст запроса
query_d_1 = f'''
select 
	i."name" as "сфера деятельности",
	count(v.id) as "количество вакансий"
from employers_industries ei 
join industries i on ei.industry_id = i.id 
join employers e on ei.employer_id = e.id 
join vacancies v on e.id = v.employer_id 
group by i.id 
order by "количество вакансий" desc 
limit 10
'''

df = pd.read_sql_query(query_d_1, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\8309179.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,сфера деятельности,количество вакансий
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585
5,Кадровые агентства,1143
6,Консалтинговые услуги,1137
7,Интернет-провайдер,1095
8,Розничная сеть (продуктовая),1034
9,Фиксированная связь,1028


2. Посмотрим среднюю зп по Сферам деятельности. ТОП 20

In [605]:
# текст запроса
query_d_2 = f'''
select 
	i."name" as "сфера деятельности",
	round(avg(coalesce ((v.salary_from + v.salary_to)/2 , v.salary_from, v.salary_to)),0) AS "ЗП"
from employers_industries ei 
join industries i on ei.industry_id = i.id 
join employers e on ei.employer_id = e.id 
join vacancies v on e.id = v.employer_id 
group by i.id 
order by "ЗП" desc NULLS LAST 
limit 20
'''

df = pd.read_sql_query(query_d_2, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\3208359040.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,сфера деятельности,ЗП
0,"Агрохимия (продвижение, оптовая торговля)",381429.0
1,Агрохимия (производство),339875.0
2,Игорный бизнес,191905.0
3,Корма для животных (производство),168300.0
4,Зоомагазин,167613.0
5,Кадровые агентства,144530.0
6,"Лакокрасочная продукция, сырье (производство)",142803.0
7,"Бытовая химия, парфюмерия, косметика (произво...",130120.0
8,"Лакокрасочная продукция, сырье (продвижение, о...",128445.0
9,"Дорожно-строительная техника, сельскохозяйстве...",122878.0


3. Посмотрим на количество ключевых навыков в зависимости от опыта работы по вакансиям DS

In [606]:
query_d_3 = f'''
select 
	v.experience "опыт работы",
	avg(length(v.key_skills) - length(replace(v.key_skills,chr(9),''))+1) as "ср. количество ключевых навыков"
from vacancies v 
where v."name"  ilike any (array[{vacancies_str}]) 
	or (v."name"  not ilike '%HTML%' and v."name" ilike '%ML%')
group by v.experience
order by "ср. количество ключевых навыков" desc 
'''
df = pd.read_sql_query(query_d_3, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\3128990646.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,опыт работы,ср. количество ключевых навыков
0,Нет опыта,7.423077
1,От 1 года до 3 лет,6.795122
2,От 3 до 6 лет,6.298077
3,Более 6 лет,6.023810


4. Определим компании, которые предлагают больше всего вакансий DS. ТОП 10.

In [607]:

query_d_4 = f'''
select
    e.name as "компания",
    count(v.id) as "количество вакансий"    
from vacancies v
join employers e ON v.employer_id = e.id
where v."name"  ilike any (array[{vacancies_str}]) 
	or (v."name"  not ilike '%HTML%' and v."name" ilike '%ML%')
group by e.name
order by "количество вакансий" desc
limit 10
'''
df = pd.read_sql_query(query_d_4, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\130630729.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,компания,количество вакансий
0,СБЕР,39
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,Бэнкс Софт Системс,6


5. Посмотрим, как влияет на зарплату DS тип рабочего графика в разрезе городов

In [608]:

query_d_5 = f'''
select
	a."name" as area_name,
	v.schedule  as schedule,
	count(v.employment) as vacancie_cnt,
	round(avg(coalesce ((v.salary_from + v.salary_to)/2 , v.salary_from, v.salary_to)),0) AS salary
from vacancies v
join areas a on a.id = v.area_id 
where (v."name"  ilike any (array[{vacancies_str}]) 
	or (v."name"  not ilike '%HTML%' and v."name" ilike '%ML%'))
	and a."name" in ({cities})
group by area_name, schedule

union 

(select
	'Другие регионы' as area_name,
	v.schedule  as schedule,
	count(v.employment) as vacancie_cnt,
	round(avg(coalesce ((v.salary_from + v.salary_to)/2 , v.salary_from, v.salary_to)),0) AS salary
from vacancies v
join areas a on a.id = v.area_id 
where (v."name"  ilike any (array[{vacancies_str}]) 
	or (v."name"  not ilike '%HTML%' and v."name" ilike '%ML%'))
	and a."name" not in ({cities})
group by schedule)
order by salary desc NULLS LAST , area_name, schedule 
'''

df = pd.read_sql_query(query_d_5, connection)
df

C:\Users\NCHirkov\AppData\Local\Temp\ipykernel_2032\2409866333.py:30: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area_name,schedule,vacancie_cnt,salary
0,Ростов-на-Дону,Удаленная работа,3,337500.0
1,Другие регионы,Удаленная работа,45,303923.0
2,Казань,Удаленная работа,6,285500.0
3,Москва,Полный день,169,220316.0
4,Санкт-Петербург,Гибкий график,9,215000.0
5,Москва,Гибкий график,11,210000.0
6,Москва,Удаленная работа,45,200000.0
7,Новосибирск,Полный день,13,190000.0
8,Санкт-Петербург,Полный день,49,179714.0
9,Санкт-Петербург,Удаленная работа,11,175346.0


In [609]:

fig = px.bar(
    df,
    x='salary', 
    y='area_name', 
    barmode='group',
    color='schedule',
    orientation='h',
    title='Зависимость заработной платы от типа трудоустройства в разрезе городов',
    labels={"schedule": "Тип рабочего графика"}
)
fig.update_layout(
    xaxis_title="средний уровень ЗП",
    yaxis_title="регион"
)
fig.show()
fig.write_html("plotly/project_02_d_5.html")

ModuleNotFoundError: No module named 'plotly.validators.layout._hoversubplots'

**ВЫВОДЫ по дополнительному исследованию:**
1) По предлагаемой средней ЗП для DS лидирует Москва (134140 руб), на втором и третьем месте Новосибирск (107777 руб.) и Санкт-Петербург(104410 руб.). Наименьшая средняя ЗП в городах-миллионерах в Волгограде (68689 руб.).
2) Если смотреть по сферам деятельности, то Разработка программного обеспечения а 13 месте (117659 руб.), два первых места у Агрохимия (продвижение, оптовая торговля)(381429 руб.) и Агрохимия (производство)(339875 руб.), на втором месте Игровой бизнес (191905 руб.). 
3) Количество клчевых навыков оказались совсем не показательными, т.к. в вакансиях где не требуется опыта более всего в среднем затребовано навыков.
4) Больше всего вакансий DS  таких компаний, как СБЕР(39), Bell Integrator (25), Банк ВТБ (ПАО) (18)
5) В разрезе влияния типа трудоустройства на среднюю ЗП можно видеть тенденцию, что есть вакансии с высокой ЗП для удаленной работы. Особенно аномально смотрятся предложения в Ростове-на-Дону. В некоторых регионах предлагается только удаленная работа. Тенденция найма сотрудника для удаленной работы должна сильно изменить предлагаемый уровень ЗП в регионах, т.к. DS вполне может удаленно работать из любого региона.
 